In [1]:
from __future__ import unicode_literals

import os

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
def parse_page(x):
    x = x.split('_')
    return ' '.join(x[:-3]), x[-3], x[-2], x[-1]


def nan_fill_forward(x):
    for i in range(x.shape[0]):
        fill_val = None
        for j in range(x.shape[1] - 3, x.shape[1]):
            if np.isnan(x[i, j]) and fill_val is not None:
                x[i, j] = fill_val
            else:
                fill_val = x[i, j]
    return x

In [3]:
df = pd.read_csv(
    '/Users/shuyangdu/Desktop/CS229/FinalProject/data/raw_data/train_2.csv', 
    encoding='utf-8'
)

In [4]:
date_cols = [i for i in df.columns if i != 'Page']

df['name'], df['project'], df['access'], df['agent'] = zip(*df['Page'].apply(parse_page))

In [5]:
df.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10,name,project,access,agent
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,16.0,27.0,29.0,23.0,54.0,38.0,2NE1,zh.wikipedia.org,all-access,spider
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,54.0,25.0,26.0,23.0,13.0,81.0,2PM,zh.wikipedia.org,all-access,spider
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,4.0,7.0,3.0,4.0,7.0,6.0,3C,zh.wikipedia.org,all-access,spider
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,6.0,16.0,19.0,30.0,38.0,4.0,4minute,zh.wikipedia.org,all-access,spider
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.0,23.0,28.0,14.0,8.0,7.0,52 Hz I Love You,zh.wikipedia.org,all-access,spider


In [6]:
le = LabelEncoder()
df['project'] = le.fit_transform(df['project'])
df['access'] = le.fit_transform(df['access'])
df['agent'] = le.fit_transform(df['agent'])
df['page_id'] = le.fit_transform(df['Page'])

In [7]:
df.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10,name,project,access,agent,page_id
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,27.0,29.0,23.0,54.0,38.0,2NE1,8,0,1,1797
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,25.0,26.0,23.0,13.0,81.0,2PM,8,0,1,1801
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,7.0,3.0,4.0,7.0,6.0,3C,8,0,1,1904
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,16.0,19.0,30.0,38.0,4.0,4minute,8,0,1,2012
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,28.0,14.0,8.0,7.0,52 Hz I Love You,8,0,1,2039


In [8]:
path = '/Users/shuyangdu/Desktop/CS229/FinalProject/data/processed'
if not os.path.isdir(path):
    os.makedirs(path)

In [9]:
df[['page_id', 'Page']].to_csv(os.path.join(path, 'page_ids.csv'), encoding='utf-8', index=False)

In [10]:
np.save(os.path.join(path, 'project.npy'), df['project'].values)
np.save(os.path.join(path, 'access.npy'), df['access'].values)
np.save(os.path.join(path, 'agent.npy'), df['agent'].values)
np.save(os.path.join(path, 'page_id.npy'), df['page_id'].values)

In [13]:
data = df[date_cols].values
np.save(os.path.join(path, 'data.npy'), np.nan_to_num(data))
np.save(os.path.join(path, 'is_nan.npy'), np.isnan(data).astype(int))

In [11]:
test_data = nan_fill_forward(df[date_cols].values)

In [14]:
np.save(os.path.join(path, 'test_data.npy'), np.nan_to_num(test_data))
np.save(os.path.join(path, 'test_is_nan.npy'), np.isnan(test_data).astype(int))

In [17]:
np.isnan(np.nan_to_num(test_data)).any()

False

In [22]:
df['page_id'].isnull().any()

False